# Lab 3 — recommender systems

In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np
from operator import itemgetter
%matplotlib inline
plt.style.use("ggplot")

What does the data look like?

In [ ]:
!hdfs dfs -cat /ix/ml-20m/ratings.txt | tail -n 2

In [ ]:
data = sc.textFile("/ix/ml-20m/ratings.txt").map(json.loads)
user = data.map(itemgetter('userId'))
user = user.map(lambda x: (x,1))
user = user.reduceByKey(lambda a,b: a+b)
item = data.map(itemgetter('movieId'))
item = item.map(lambda x: (x,1))
item = item.reduceByKey(lambda a,b:a+b)
user = user.map(lambda x:x[1])

In [ ]:
print(data.filter(lambda x: x['userId'] == 1).count())

In [ ]:
print(user.count())
x = (user.collect())
y = (item.collect())
plt.plot(sorted(x))
plt.show()
plt.plot(y[1])
plt.show()

print(x[0])


In [ ]:
%run rate-movies.py

In [ ]:
!hdfs dfs -put my-ratings.txt

In [ ]:
myRates = sc.textFile('my-ratings.txt').map(json.loads)
newData = data.union(myRates)
trainingSet = data.filter(lambda x: x['timestamp']%10 <= 7)
validationSet = data.filter(lambda x: x['timestamp']%10 > 7)
mean = trainingSet.map(lambda x: x['rating']).mean()



In [ ]:
print(mean)
userBias = trainingSet.map(lambda x:(x['userId'],x['rating']-mean)).groupByKey().map(lambda x:(x[0],sum(x[1])/len(x[1]))).sortByKey().collect()

itemBias = trainingSet.map(lambda x: (x['movieId'],x['rating']-mean )).goupByKey().sortByKey()#.map(lambda x:(x[0],sum(x[1])/len(x[1]))).sortByKey()

print(itemBias.take(5))


In [ ]:
a= [1,3,5,8,9,45]
print([t for t in a if t==2][0] if 2 in a else 0)